# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [96]:
# Details Student 1:

# Name: Nitay Hen
# ID:   207234253
# Email: nitaikind@gmail.com

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [97]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [98]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [99]:
!pip install wn
!python -m wn download omw-he:1.4

Defaulting to user installation because normal site-packages is not writeable



Cached file found: C:\Users\Nitay Hen\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\NITAYH~1\AppData\Local\Temp\tmpwgk19vd6\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [100]:
from nltk.corpus import wordnet as wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [101]:
pip install hebrew_tokenizer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [102]:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [103]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [104]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [105]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [115]:
#create a token list with hebrew words only
def BuildTokenList(str):
    tokens = ht.tokenize(str)
    lst=[]
    for grp, token, token_num, (start_index, end_index) in tokens:
        if grp=="HEBREW":
            lst.append(token)
            
    return lst

In [116]:
#return majority vote between the 5 models used, if male or female
def BuildMetaList(list1,list2,list3,list4,list5):
    lst=[]
    for i, j, k,l,m in zip(list1, list2, list3,list4,list5):
        if i+j+k+l+m>0:
            lst.append(1)
        else:
            lst.append(-1)
            
    return lst

In [117]:
#creating a model made of 5 different classifying algorithms and put them in a dictionary
def BuildModel(X_train,y_train):
    #create pipelines
    dct={}
    dct['text_clf_nb']=Pipeline([('vect', CountVectorizer(tokenizer=BuildTokenList, max_df=0.9)),('clf_nb', MultinomialNB())])
    dct['text_clf_svc']=Pipeline([('vect', CountVectorizer(tokenizer=BuildTokenList, max_df=0.9)),('svc', LinearSVC())])
    dct['text_clf_knn']=Pipeline([('vect', CountVectorizer(tokenizer=BuildTokenList, max_df=0.9)),('knn', KNeighborsClassifier())])
    dct['text_clf_tree']=Pipeline([('vect', CountVectorizer(tokenizer=BuildTokenList, max_df=0.9)),('tree', DecisionTreeClassifier())])
    dct['text_clf_SGD']=Pipeline([('vect', CountVectorizer(tokenizer=BuildTokenList, max_df=0.9)),('tree', SGDClassifier())])
    
    #feature selection, ('norm', preprocessing.Normalizer(norm='l1'))
    #Note that we want to ignore words used by 90%+ of the given paragraphs
    
    #build models
    dct['text_clf_nb'].fit(X_train.flatten(), y_train)
    dct['text_clf_svc'].fit(X_train.flatten(), y_train)
    dct['text_clf_knn'].fit(X_train.flatten(), y_train)
    dct['text_clf_tree'].fit(X_train.flatten(), y_train)
    dct['text_clf_SGD'].fit(X_train.flatten(), y_train)

    return dct

In [118]:
#predicting X set using the 5 models in the dictionary
def PredictModel(X,dct):

    docs_test = X.flatten()
    predicted_nb = dct['text_clf_nb'].predict(docs_test)
    predicted_svc = dct['text_clf_svc'].predict(docs_test)
    predicted_knn = dct['text_clf_knn'].predict(docs_test)
    predicted_tree = dct['text_clf_tree'].predict(docs_test)
    predicted_SGD =  dct['text_clf_SGD'].predict(docs_test)
    predicted_total = BuildMetaList(predicted_knn, predicted_nb, predicted_svc, predicted_tree, predicted_SGD)
    
    return predicted_total

In [120]:
#load train/test files
train_filename = 'annotated_corpus_for_train.csv'
test_filename = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

#giving genders numeral values
df_train.replace({'m':1,'f':-1},inplace=True)
df_test.replace({'m': 1, 'f': -1}, inplace=True)

#build and predict on train data
X = df_train.drop('gender', axis=1).values
y = df_train['gender'].values
dct=BuildModel(X,y)
predicted_total=PredictModel(X,dct)

#f1 results / accuracy rate of the model
print("f1: ", f1_score(predicted_total, y, average='macro'))
f1_result = f1_score(predicted_total, y, average='macro')
percentage = "{:.1%}".format(f1_result)
print("Prediction accuracy: ", percentage)
y_train_labels = ['m' if label == 1 else 'f' for label in y]
predicted_total_labels = ['m' if pred == 1 else 'f' for pred in predicted_total]
df_res = pd.DataFrame({'y_train': y_train_labels, 'y_pred': predicted_total_labels})
df_res.head(10)
df_res.tail(10)
df_res.shape

#predict y test set and create dataframe
X_test=df_test.drop('test_example_id', axis=1).values
predicted_total=PredictModel(X_test,dct)
df_predicted=pd.DataFrame({'test_example_id':df_test.test_example_id,'predicted_category':predicted_total})
    
#reverting numbers back to genders
df_predicted['predicted_category'].replace({1: 'm', -1: 'f'}, inplace=True)

f1:  0.9981642787663368
Prediction accuracy:  99.8%


,y_train,y_pred
0,m,m
1,m,m
2,f,f
3,m,m
4,f,f
5,f,f
6,m,m
7,f,f
8,m,m
9,f,f


,y_train,y_pred
743,f,f
744,m,m
745,f,f
746,m,m
747,m,m
748,m,m
749,m,m
750,m,m
751,f,f
752,m,m


(753, 2)

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [111]:
df_predicted.to_csv('classification_results.csv',index=False)